In [ ]:
######   feature 2

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Job Skills Analysis") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/23 10:52:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
df = spark.read.csv("job_descriptions.csv", header=True, inferSchema=True)

In [15]:
filtered_df = df.filter(df["Role"].like("%Interaction Designer%"))

In [11]:
from pyspark.sql.functions import explode, split, lower,col,desc

# Splitting the skills string into individual skills and converting to lowercase
exploded_skills = filtered_df.withColumn("Skill", explode(split(lower(col("Skills")), ", ")))

# Grouping by skills and counting occurrences
skills_count = exploded_skills.groupBy("Skill").count()

# Sorting by the most frequently mentioned skills
most_common_skills = skills_count.orderBy(desc("count"))


In [12]:
most_common_skills.show()

+--------------------+-----+
|               Skill|count|
+--------------------+-----+
|javascript fronte...| 3563|
|               react| 3563|
|                html| 3563|
|                 css| 3563|
|angular) user exp...| 3563|
|python) database ...| 3452|
|             node.js| 3452|
|                 sql| 3452|
|server-side progr...| 3452|
|            mongodb)| 3452|
+--------------------+-----+



In [6]:
from pyspark.sql.functions import col, explode, lower, regexp_replace, split

# Normalize and clean up the skills text
# Replace specific punctuations with a comma, handle and normalize spaces
normalized_skills_df = df.withColumn("normalized_skills", 
    regexp_replace(col("Skill"), r"\)\s+", "), ").replace(["(", ")"], "", "normalized_skills")
)

# Splitting the 'normalized_skills' column on commas and exploding into individual skills
exploded_skills = normalized_skills_df.withColumn("Skill", explode(split(col("normalized_skills"), ",\s*")))

# Further cleaning to trim spaces and convert to lowercase
cleaned_skills = exploded_skills.withColumn("Skill", lower(trim(col("Skill"))))

# Grouping by skills and counting occurrences
skills_count = cleaned_skills.groupBy("Skill").count()

# Sorting by the most frequently mentioned skills
most_common_skills = skills_count.orderBy(col("count").desc())


TypeError: 'Column' object is not callable

In [16]:
from pyspark.sql.functions import col, explode, lower, trim, regexp_replace, split
# This replaces ')' followed by spaces with '), ' to maintain separation after exploding
df = filtered_df.withColumn("normalized_skills", regexp_replace(col("Skills"), r"\)\s+", "), "))

# Explode the skills based on ', ' after normalizing it to handle various formats
df = df.withColumn("Skill", explode(split(col("normalized_skills"), ",\s*")))

# Cleaning up skills by trimming spaces and converting to lowercase for uniformity
df = df.withColumn("Skill", lower(trim(col("Skill"))))

# Count the occurrences of each skill
skill_counts = df.groupBy("Skill").count().orderBy(col("count").desc())


In [17]:
skill_counts.show()

+--------------------+-----+
|               Skill|count|
+--------------------+-----+
|interaction desig...|20580|
+--------------------+-----+



In [ ]:
######### feature 3

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, regexp_replace, array_contains, split
from pyspark.sql.types import BooleanType

df = spark.read.csv("job_descriptions.csv", header=True, inferSchema=True)

# User profile
user_profile = {
    "skills": ["HTML", "CSS", "JavaScript", "React", "Angular", "UX"],
    "salary": 100000
}

# Normalize skills in the dataset
df = df.withColumn("Normalized_Skills", lower(regexp_replace(col("Skills"), "[^a-zA-Z0-9\s]", "")))


In [21]:
# Function to check if any user skill is in job skills
from pyspark.sql.functions import udf, col, lower, regexp_replace, array_contains, split, array

def contains_skill(job_skills, user_skills):
    job_skills_list = job_skills.split(" ")
    return any(skill.lower() in job_skills_list for skill in user_skills)

contains_skill_udf = udf(contains_skill, BooleanType())

# Apply the UDF to filter jobs by skills
df_filtered_by_skills = df.filter(contains_skill_udf(col("Normalized_Skills"), array(user_profile['skills'])))


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `HTML` cannot be resolved. Did you mean one of the following? [`Role`, `Job Id`, `skills`, `Company`, `Contact`].;
'Filter contains_skill(Normalized_Skills#605, array('HTML, 'CSS, 'JavaScript, 'React, 'Angular, 'UX))#631
+- Project [Job Id#559L, Experience#560, Qualifications#561, Salary Range#562, location#563, Country#564, latitude#565, longitude#566, Work Type#567, Company Size#568, Job Posting Date#569, Preference#570, Contact Person#571, Contact#572, Job Title#573, Role#574, Job Portal#575, Job Description#576, Benefits#577, skills#578, Responsibilities#579, Company#580, Company Profile#581, lower(regexp_replace(Skills#578, [^a-zA-Z0-9\s], , 1)) AS Normalized_Skills#605]
   +- Relation [Job Id#559L,Experience#560,Qualifications#561,Salary Range#562,location#563,Country#564,latitude#565,longitude#566,Work Type#567,Company Size#568,Job Posting Date#569,Preference#570,Contact Person#571,Co

In [8]:
from pyspark.sql.functions import udf, col, lower, trim, regexp_replace, array, lit
from pyspark.sql.types import BooleanType
import re

# UDF to check if any user skill is in job skills
def contains_skill(job_skills, user_skills):
    job_skills_list = job_skills.split(" ")
    return any(skill.lower() in job_skills_list for skill in user_skills)

contains_skill_udf = udf(contains_skill, BooleanType())

# Prepare a list of user skills as literal columns for comparison
user_skills = [lit(skill.lower()) for skill in user_profile['skills']]

# Normalize skills in the dataset
df = df.withColumn("Normalized_Skills", lower(trim(regexp_replace(col("Skills"), "[^a-zA-Z0-9\s]", ""))))

# Filter jobs based on skills match
df_filtered_by_skills = df.filter(contains_skill_udf("Normalized_Skills", array(user_skills)))

# Assuming a function or method to parse the salary
# Function to parse salary and check range
def salary_in_range(salary_range, desired_salary):
    # Remove dollar signs and 'K', and split by '-'
    salary_min, salary_max = [int(s) * 1000 for s in re.sub(r'[^\d-]', '', salary_range).split('-')]
    return salary_min <= desired_salary <= salary_max

salary_udf = udf(salary_in_range, BooleanType())

# Filter jobs where the salary fits the user's desired salary
df_final = df_filtered_by_skills.filter(salary_udf("Salary Range", lit(user_profile["salary"])))

# Select and display recommended job roles
df_final.select("Job Title").distinct().show()


+-------------------+
|          Job Title|
+-------------------+
|   Graphic Designer|
|      Web Developer|
|Front-End Developer|
| Software Developer|
| Front-End Engineer|
|       UI Developer|
|  Software Engineer|
+-------------------+



In [9]:
import json

# Assuming df_final contains the filtered DataFrame based on salary and skills
job_titles = df_final.select("Job Title").distinct().rdd.flatMap(lambda x: x).collect()

# Construct the output dictionary
output_dict = {
    "roles": job_titles
}

# If you need a JSON string
json_output = json.dumps(output_dict, indent=4)
print(json_output)

{
    "roles": [
        "Graphic Designer",
        "Web Developer",
        "Front-End Developer",
        "Software Developer",
        "Front-End Engineer",
        "UI Developer",
        "Software Engineer"
    ]
}
